In [1]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json

sys.path.append(str(Path("..").resolve()))
from src import *

# Seeding Missing Collections and Entries
ℹ️ This notebook require the execution of [`1.1-Migrating_MySQL_To_MongoDB`](1.1-Migrating_MySQL_To_MongoDB.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `2_migration_preprocessed` snapshot.

In [2]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/2_migration_preprocessed/omero_museum"

In [3]:
schema=json.load(open("schema.json"))
connector=MongoDBConnector("omero_museum")
db=connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]:
[artworks]: _id date description room type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]:
[messages]:
[rooms]:
[suppliers]:
[surveys]: _id accompanying_persons_visit date_of_compilation evaluation_of_experience evaluation_of_facility evaluation_of_visit number_of_visits reason_for_visit return title_of_studies type_of_visit
[tickets]: _id date price
[visitors]: _id cell phone number email fare surname
[workshops]: _id duration price_class price_person type


## Generating missing entries


In [27]:
s=EntrySeeder("iban")
s.seed()

'IT15G0912187133359675929091'